# 서울시 평균 기온 예측 Baseline Code  

## 전체 과정
1. **데이터 불러오기**
   - `train.csv`와 `sample_submission.csv` 파일을 불러옵니다.
     - `train.csv` 파일에는 과거의 기상 데이터와 평균 기온이 포함되어 있습니다. 
     - `sample_submission.csv` 파일에는 예측할 날짜와 기온 예측값을 채워 넣을 형식이 포함되어 있습니다.  
2. **데이터 전처리**    
   - train_df의 일시 컬럼의 자료형을 datetime 으로 변경합니다.  
   - 일시 컬럼을 인덱스로 설정합니다.  
   - index의 freq 속성을 'D'로 변경합니다.  
3. **ARIMA 모델 설정 및 학습**
   - ARIMA 모델을 생성합니다.
   - train_df의 평균기온 컬럼을 사용하여 ARIMA 모델을 학습합니다.  
4. **예측 수행**
   - submission_df의 일시 컬럼의 자료형을 datetime으로 변경합니다.  
   - submission_df에서 일시 컬럼의 최솟값과 최댓값을 설정하여 시작날짜와 종료날짜를 지정합니다.  
   - 시작날짜와 종료날짜를 지정하여 기간에 대한 ARIMA 모델의 예측값을 반환받습니다.    
5. **결과 저장**
   - 학습된 모델을 사용하여 submission_df의 기간에 대한 평균기온을 예측합니다.    
   - 예측된 기온을 `sample_submission.csv` 파일 형식에 맞추어 저장합니다.
   - 저장된 파일은 `baseline_submit.csv`와 같은 형태로 제출할 수 있습니다.

# 1. 데이터 불러오기
- `train.csv` 파일은 다음과 같은 컬럼을 포함하고 있습니다.
    - 일시: 과거 데이터들의 관측 날짜 (1960-01-01 ~ 2022-12-31)
    - 최고기온, 최저기온, 일교차
    - 강수량, 평균습도, 평균풍속
    - 일조합, 일사합, 일조율
    - 평균기온: 우리가 예측하고자 하는 대상

    <br>
- `sample_submission.csv` 파일에는 다음과 같은 컬럼이 포함되어 있습니다.
    - 일시: 문제에서 정의된 미래에 대해 예측해야 할 날짜 (2023-01-01 ~ 2023-12-24)
    - 평균기온: 이 값에 대한 예측을 생성해야 합니다    
    
이번 베이스라인에서는 ARIMA 모델을 사용한 단변량예측을 수행합니다.    

In [2]:
import pandas as pd

# 파일 경로
train_path = './train.csv' # 학습 데이터 파일 경로
submission_path = './sample_submission.csv' # 제출 양식 파일 경로

# 데이터 불러오기
train_df = pd.read_csv(train_path)
submission_df = pd.read_csv(submission_path)

display(train_df.head())
display(submission_df.head())

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
0,1960-01-01,2.2,-5.2,7.4,NaN,68.3,1.7,6.7,NaN,NaN,-1.6
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,NaN,NaN,-1.9
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,NaN,NaN,4.0
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,NaN,NaN,7.5
4,1960-01-05,1.3,-8.2,9.5,NaN,44.0,5.1,8.2,NaN,NaN,-4.6


,일시,평균기온
0,2023-01-01,0
1,2023-01-02,0
2,2023-01-03,0
3,2023-01-04,0
4,2023-01-05,0


# 2. 데이터 전처리
- ARIMA 모델에 적합하도록 데이터를 가공합니다.

In [3]:
# 날짜 데이터 변환
train_df['일시'] = pd.to_datetime(train_df['일시'])
train_df = train_df.set_index('일시')

# 데이터의 시간 간격 지정
train_df.index.freq = 'D'

# 일시 컬럼이 인덱스로 할당됩니다.  
train_df.head()

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
일시,,,,,,,,,,
1960-01-01,2.2,-5.2,7.4,NaN,68.3,1.7,6.7,NaN,NaN,-1.6
1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,NaN,NaN,-1.9
1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,NaN,NaN,4.0
1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,NaN,NaN,7.5
1960-01-05,1.3,-8.2,9.5,NaN,44.0,5.1,8.2,NaN,NaN,-4.6


# 3. ARIMA 모델 설정 및 학습
- ARIMA 모델을 생성하고, 학습 데이터로 모델을 학습합니다.
- ARIMA 모델은 시계열 데이터 분석에 사용하는 통계적 모델 중 하나로,    
정상성을 충족시키면서 적절한 파라미터를 선택할 경우 좋은 성능을 보입니다.    

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# ARIMA 모델 훈련
model = ARIMA(train_df['평균기온'], order=(2, 1, 3))  # p, d, q 값은 조정 필요
model_fit = model.fit()

# 4. 예측 수행
- 학습된 모델을 사용하여 미래 기온을 예측합니다.

In [9]:
submission_df['일시'] = pd.to_datetime(submission_df['일시'])

# 예측할 기간 설정
start_date = submission_df['일시'].min()
end_date = submission_df['일시'].max()

# ARIMA 모델을 사용하여 예측
forecast = model_fit.predict(start=start_date, end=end_date, typ='levels')

# 5. 예측 결과 저장
- 예측된 결과를 sample_submission.csv 파일 형식에 맞추어 저장합니다.

In [14]:
# 예측 결과 추출 및 확인
submission_df['평균기온'] = forecast.values
display(submission_df.head())

# 예측 결과 저장
submission_df.to_csv('./baseline_submit.csv', index=False)

,일시,평균기온
0,2023-01-01,-2.923068
1,2023-01-02,-4.701615
2,2023-01-03,-5.781839
3,2023-01-04,-6.478585
4,2023-01-05,-6.964074
